This project ask you to perform sentiment analysis on 10-K Financial Reports.

1. You are asked to pick 1 firm, download the entire history of 10-K and 10-Q financial reports from the SEC Edgar Server.

2. Perform sentiment analysis on all the 10-K and 10-Q reports.

3. Merge with stock return data.

4. Determine if changes in negative sentiment can predict stock returns.

# Final Project: Sentiment Analysis of 10-K and 10-Q

## Obtain 10-K Financial Reports

In [1]:
pip install -U sec-edgar-downloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.1 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=3087c212f5b8a6cfd7784cc5514aa9f99f2e586ff7dda7c84e402e50e720186f
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [2]:
from sec_edgar_downloader import Downloader
dl = Downloader("/Users/carlotaginer/Downloads/")

In [3]:
dl.get("10-K", "MSFT")

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Skipping filing detail download for '0001032210-00-001961' due to network error: 404 Client Error: Not Found for url: https://www.sec.gov/Archives/edgar/data/789019/000103221000001961/0001.txt.


23

## Analysis 

Run tests to examine whether changes in sentiment predicts future stock returns.

Cumulative 5 days, 10 days, 1 month returns
    
Negative sentiment vs positive sentiment vs overall sentiment
        

In [5]:
# Opening the company file
index_file = open("company.idx").readlines()

#confirming the header of the file
print(index_file[:10])

FileNotFoundError: ignored

In [ ]:
# I will keep the information from this line in a list called find_list

find_list = []
item = 0
line = 0

while item < 100:
i = index_file[line]
loc1 = i.find('10-K')
loc2 = i.find("NT 10-K") 
loc3 = i.find("10-K/A")

#I will strictly keep 10-K files, not NT 10-K or 10-K/A
if (loc2 == -1) and (loc1 != -1) and (loc3 == -1):
    find_list.append(i)
line+=1
item = len(find_list)

# Sanity check: if the command worked properly, the list should have 100 items.
print(len(find_list))

In [6]:
# commands below will split the line, and send the links to a list called ReportList, and the CIK+date issued to a list called Company_No (this will be the name of the file when download)
ReportList = []
Company_No = []
for i in find_list:
    split_i = i.split()
    ReportList.append("https://www.sec.gov/Archives/" + split_i[-1])
    Company_No.append(split_i[-3] + "_" + split_i[-2])
print(ReportList[:5])
print(Company_No[:5])


NameError: ignored

In [ ]:
import os
os.chdir("C:/Users/carlotaginer/Desktop/Open Code/10K Text Mining/10K Files")

def createfile(filename, content):
    name= filename + ".txt"  # Here I define the name of the file
    with open(name, "w") as file:
        file.write(str(content)) # Here I define its content, which will be the textual content from the 10-K files.
        file.close()
        print("Succeed!")

In [ ]:
company_order = 0
unable_request = 0

for a_index in range(100):
    web_add = ReportList[a_index]
    filename = Company_No[a_index]

    webpage_response = requests.get(web_add, headers={'User-Agent': 'Mozilla/5.0'}) 

    if webpage_response.status_code == 200: 
        # The HTTP 200 OK success status response code indicates that the request has succeeded. 
        body = webpage_response.content
        createfile(filename, body)
    else:
        print ("Unable to get response with Code : %d " % (webpage_response.status_code))
        unable_request += 1

    a_index +=1

print(unable_request) 

In [ ]:
 # First, I will create a list with the name of the files in the folder where I downlaoded the 10-K files
os.chdir("C:/Users/carlotaginer/Desktop/Open Code/10K Text Mining/10K Files")
all_files = os.listdir()

# I will use only 10 random files from the 100 we downloaded.
import random

# I create a list with 10 random indexes between 0 and 100 to selected the files in the folder.
random_index = []
while len(random_index) < 10:
    n = random.randint(0,100)
    if n not in random_index:
        random_index.append(n)

# Here I create a new list with the 10 chosen files.
chosen_file = [all_files[i] for i in random_index]
print(chosen_file)

In [ ]:
def read_txt(file_name):
    txt_file = open(file_name,"r",encoding='UTF8')                                       
    str_txt = txt_file.read()
    return str_txt

In [ ]:

import re
text_start_pattern = re.compile(r'<DOCUMENT>') 
text_end_pattern = re.compile(r'</DOCUMENT>')
type_pattern = re.compile(r'<TYPE>10-K[^\n]+')

# Here I will define a function that will be used to extract the textual data from 10-K txt files.
def textual_content(file):
    doc_start_list = [x.start() for x in text_start_pattern.finditer(file)] #assigns the first index from the starting pattern created before
    oc_end_list = [x.end() for x in text_end_pattern.finditer(file)] #assigns the last index from the ending pattern created before
    type_list = [x[len('<TYPE>'):] for x in type_pattern.findall(file)] #assigns the type of the documents, which will always be 10-K's because we restricted it before
    for doc_type, start_index, end_index in zip(type_list, doc_start_list, doc_end_list):
        report_content = file[start_index:end_index]
    return report_content


In [ ]:
text_initial = read_txt(chosen_file[0])
text_10k = textual_content(text_initial)
print(text_10k[0:30])
print(text_10k[-30:])

In [ ]:
from bs4 import BeautifulSoup
def BeautifulSoup_clean1(str_txt):
    soup = BeautifulSoup(str_txt,'html.parser')
    return soup.get_text()


In [ ]:
import string
from nltk.tokenize import word_tokenize

def further_clean(text):
    

# This function replaces it by an empty space on the text 
    for a_sign in ['\\n', '\\t', '☐', '☒', '\xa0', '●', '“', '”']:
        text = text.replace(a_sign," ")

# Here, for each punctution in a set of all existing punctuations, the function also replaces it by an empty space.
    for a_punc in string.punctuation:
        text = text.replace(a_punc, " ")


text = re.sub('\s+'," ", text).lower()

return text.strip() 


In [ ]:
 # Tokenizing and counting the total frequency of words

from nltk.tokenize import word_tokenize

def word_count(text):
    word_list = word_tokenize(text) #splits the text into words and assigns it to a new list
    total_num = len(word_list) #count the number of words in the new list
    return total_num, word_list

In [ ]:
import pandas as pd 

LM_negw = pd.read_csv("C:/Users/carlotaginer/Desktop/Open Code/10K Text Mining/LM_negative_wordlist.csv")

# Transforming the file into a list of negative words, and converting all entries to lowercase.
LM_negw = list(LM_negw['Negative_word'])
LM_negw = [j.lower() for j in LM_negw]

# I will create a dictionary that assigns the number of appearances to each negative words

negw_dic = {} #creates an empty dictionary
for A_NegWord in LM_negw:
    negw_dic[A_NegWord] = 0 # For each word in the LM_negw list, we create an entry inside the dictonary with the word and the number 0.


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/carlotaginer/Desktop/Open Code/10K Text Mining/LM_negative_wordlist.csv'

In [ ]:
# I will redefine the function to tokenize the text and will add the steps to count the negative words.

from nltk.tokenize import word_tokenize

def word_count(text):

    # Tokenizing and counting total words
    word_list = word_tokenize(text) # Splits the text into words and assigns it to a new list
    total_num = len(word_list) # Count the number of words in the new list


    # Counting negative words

    negw_dic_ex = negw_dic.copy() # This commands creates a copy of the dictionary we created with the L&M negative word list

    # This loops checks if each word in the text is a negative word, and if so, adds 1 to the respective entry in 
    # the "negw_dic_ex" dictionary.

    for a_word in word_list:
        if a_word in LM_negw:
            negw_dic_ex[a_word] += 1


    # This commands creates a new variable from a dictionary (the negw_dic_ex created before), and the orient="index" means that
    # the keys of the dictionary will be the rows of the variable.

    negw_df = pd.DataFrame.from_dict(negw_dic_ex, orient='index')
    negw_df.reset_index(inplace=True) # This command modifies the indexes of the list, converting them to default indexes.
    # This command renames the columns of the variable (inplace=True means that the variable will be modified without creating a new variable)
    negw_df.rename(columns={"index": "Negative_words", 0: "Word_counts"}, inplace=True)


    total_neg = sum(list(negw_dic_ex.values()))  # Total number of negative words
    negw_fre = total_neg/total_num # % of negative words in the text

    return {"Negative_WordCount_df":negw_df, "Total_WordNumber":total_num, "Total_Negative_WordCount": total_neg,
        "Negative_WordFrequency": negw_fre}

In [ ]:
# First, we open 4 empty lists to append the 4 returns from the function we've just defined.
Neg_WordDf_list = []
Total_Word_list = []
Total_Neg_Word_list = []
Frequency_list = []

# For each file in the chosen_file list (the list with the 10-K txt files)
for a_file in chosen_file:
    text = read_txt(a_file) #run the read_txt fuction that opens the txt file
    text = textual_content(text) #run the textual_content function to get the textual content from the file opened before.
    text = BeautifulSoup_clean1(text) #run the BeautifulSoup_clean1 function to clean the data even better using the html.parser algorithm. 
    text = further_clean(text) #run the further_clean function to remove symbols, punctiations, and convert strings to lower case. 
    result = word_count(text) #run the word_count functions that tokenize the text, and deal with negative word counting.

    Neg_WordDf_list.append(result['Negative_WordCount_df'])
    Total_Word_list.append(result["Total_WordNumber"])
    Total_Neg_Word_list.append(result["Total_Negative_WordCount"])
    Frequency_list.append(result["Negative_WordFrequency"])

In [ ]:
# Here we create a new dictionary using data from the lists defined before.
summary = {'File_name':chosen_file, 'Total_WordNumber':Total_Word_list, 'Total_Negative_WordCount':Total_Neg_Word_list,'Negative_WordFrequency':Frequency_list}

summary_df = pd.DataFrame(summary, index=range(1,11))
summary_df